<a href="https://colab.research.google.com/github/syoooooung/capstone_design/blob/main/Query_Decomposition/%5Bfinal_form%5Dsubquery_by_conjunction_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai ragas datasets faiss-cpu groq langchain_community evaluate anls sentence_transformers
!pip install -U langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.9/173.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.

In [2]:
from google.colab import drive
from langchain.vectorstores import FAISS
import faiss
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from google.colab import drive, userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY2')

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data= [
    {
        "question": "Which American singer and songwriter has a mezzo-sprano vocal range, Tim Arm or Tori Amos?",
        "answer": "Sam Bankman-Fried",
        "decomposed": [
            "Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges?",
            "Who is accused by prosecutors of committing fraud for personal gain in the cryptocurrency industry?",
            "What industry is the individual associated with who is facing a criminal trial on fraud and conspiracy charges?",
            "Who has been reported by The Verge and TechCrunch in relation to a criminal trial on fraud and conspiracy charges?"
        ],
        "supporting_facts_sentences": [
            "Before his fall, Bankman-Fried made himself out to be the Good Boy of crypto — the trustworthy face of a sometimes-shady industry.",
            "The highly anticipated criminal trial for Sam Bankman-Fried, former CEO of bankrupt crypto exchange FTX, started Tuesday to determine whether he’s guilty of seven counts of fraud and conspiracy.",
            "The prosecution painted Bankman-Fried as someone who knowingly committed fraud to achieve great wealth, power and influence, while the defense countered that the FTX founder acted in good faith, never meant to commit fraud or steal and basically got in over his head."
        ]
    }
]
# input 파일 형태 예시

In [ ]:
%%capture
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
import os
os.kill(os.getpid(), 9)

In [7]:
import re
import json
import openai

# Load JSON data from file
input_path = "/content/selfgen_test_noqd.json"  #input 파일 경로
output_path = "/content/selfgen_s_noqd.json"  # 결과 저장 경로
model = "gpt-4o"  # Replace with your model if needed


def split_sentence(text):
    # 예외 처리할 접속사 패턴
    exception_patterns = [
        r'\b(?:both|either|neither)\b.*?\b(?:and|or|nor)\b',
        r'\b(?:not only)\b.*?\b(?:but also)\b'
    ]

    # 큰따옴표 내부의 내용을 보호하기 위해 콤마를 임시 치환
    text = re.sub(r'(".*?")', lambda match: match.group(0).replace(",", "_COMMA_"), text)

    # 예외 패턴을 먼저 처리하여 접속사 뒤에 있는 것들은 쪼개지 않도록 함
    for pattern in exception_patterns:
        text = re.sub(pattern, lambda match: match.group(0).replace(" ", "_EXCEPTION_"), text)

    # 접속사 및 콤마 기준으로 분리
    conjunctions = r'\b(?:and|or|but|so|nor)\b'
    parts = re.split(fr'\s*(?:{conjunctions}|,)\s*', text)

    # 예외 패턴에 대해서는 다시 복원
    parts = [part.replace("_COMMA_", ",").replace("_EXCEPTION_", " ") for part in parts]

    # 후처리: 큰따옴표로 시작하는 부분을 만나면 바로 앞 세션 하나와만 병합
    merged_parts = []
    i = 0
    while i < len(parts):
        part = parts[i].strip()

        # 큰따옴표로 시작하는 세션을 만났을 때
        if part.startswith('"'):
            # 앞의 세션과 병합
            if merged_parts:
                merged_parts[-1] = merged_parts[-1].strip() + ", " + part
            else:
                merged_parts.append(part)
        else:
            merged_parts.append(part)

        i += 1

    # 결과 반환
    merged_parts = [part for part in merged_parts if part]
    print(merged_parts)
    return merged_parts

Instruction_str = """
Given a query with multiple entities or phrases separated by conjunctions or commas, create each sub-sentence by keeping as much of the original phrasing as necessary to maintain the intent and meaning of the original query.

Instructions:
1. For each separated part, complete the meaning of the sentence using the previous part of the current section.
2. Preserve original terminology wherever possible, guaranteeing that each sub-sentence forms a grammatically complete sentence.
3. The number of sub-sentences created must match the number of separated parts.

"""
#######################MultihopRAG333333333333#####################
# 서브쿼리 생성 프롬프트
def generate_subqueries2(original_query, split_results):
    prompt = f"""
Original Query: "{original_query}"
Separated Parts:
{split_results}

Example:
Original Query: "Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by both The Verge and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?"
Seperated Parts: ['Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud', 'conspiracy charges', 'as reported by both The Verge and TechCrunch', 'is accused by prosecutors of committing fraud for personal gain?']
->
- subquery: Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud charges?
- subquery: Who is the individual associated with the cryptocurrency industry facing conspiracy charges?
- subquery: Who is the individual associated with the cryptocurrency industry, as reported by both The Verge and TechCrunch?
- subqeury: Who is the individual is accused by prosecutors of committing fraud for personal gain?


Example:
Original Query: "Who's birth date is the first earliest, among the birth date of Neil Armstrong, the first human to set foot on the Moon during the Apollo 11 mission in 1969, whose historic words, \"That's one small step for a man, one giant leap for mankind\", marked a significant achievement in space exploration, Taylor Swift, and Chris Hemsworth?"
Seperated Parts: ['Who's birth date is the first earliest', 'among the birth date of Neil Armstrong', 'the first human to set foot on the Moon during the Apollo 11 mission in 1969', 'whose historic words, "That's one small step for a man, one giant leap for mankind"', 'marked a significant achievement in space exploration', Taylor Swift', 'Chris Hemsworth?']
->
- subquery: Who's birth date is the first earliest?
- subquery: What is the birth date of Neil Armstrong?
- subquery: Was Neil Armstrong the first human to set foot on the Moon during the Apollo 11 mission in 1969?
- subqeury: Is historic words of Neil Armstrong \"That's one small step for a man, one giant leap for mankind\"?
- subqeury: Is historic words of Neil Armstrong mark a significant achievement in space exploration?
- subqeury: What is the birth date of Taylor Swift?
- subqeury: What is the birth date of Chris Hemsworth?


Now, based on the above examples, provide a list of subqueries for each part of the sentence while preserving the essential meaning.
Don't say anything other than the format that starts with this form (- subquery: ). And the decomposed query that is generated can't just generate the original question. Don't forget the purpose of dividing in the original question.
"""
    return prompt

# LLM 호출 예시
def get_subqueries2(original_query, model="gpt-4o"):
    split_results = split_sentence(original_query)
    prompt = generate_subqueries2(original_query, split_results)

    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": Instruction_str},
            {"role": "user", "content": prompt}
            ],
        temperature=0
    )

    # 각 서브쿼리의 앞에 있는 "subquery:"와 "-"를 제거하고 리스트로 반환
    subqueries = [
        query.replace("- subquery: ", "").strip()  # "subquery:" 제거
        for query in response.choices[0].message.content.splitlines() if query.strip()
    ]
    return subqueries

# JSON 데이터 처리 함수
def process_json(data):
    processed_entries = []
    cnt=0
    for entry in data:
        cnt+=1
        if cnt >100:
            continue
        if cnt >10:
            break

        processed_entry = {
            "question": entry["question"],
            "gt_ans": entry["gt_ans"],
            #"gt_doc": entry.get("gt_doc", []),
            "question_d": get_subqueries2(entry["question"])
        }
        processed_entries.append(processed_entry)
    return processed_entries

# Load and process JSON
with open(input_path, 'r') as file:
    data = json.load(file)

processed_data = process_json(data)

# Save the updated JSON file
with open(output_path, 'w', encoding='utf-8') as file:
    json.dump(processed_data, file, ensure_ascii=False, indent=4)

print("JSON file updated and saved successfully.")


["Who's birth date is the first earliest", 'among the birth date of Neil Armstrong', 'the first human to set foot on the Moon during the Apollo 11 mission in 1969', 'whose historic words, "That\'s one small step for [a] man, one giant leap for mankind," marked a significant achievement in space exploration', 'Taylor Swift', 'Chris Hemsworth?']
["Who's birth date is the second earliest", 'among the birth date of Neil Armstrong', 'the first human to set foot on the Moon during the Apollo 11 mission in 1969', 'whose famous words, "That\'s one small step for [a] man, one giant leap for mankind," marked a pivotal moment in space exploration', 'Taylor Swift', 'Will Smith?']
["Who's birth date is the third earliest", 'among the birth date of Neil Armstrong', 'the first human to set foot on the Moon during the Apollo 11 mission in 1969', 'whose famous words "That\'s one small step for [a] man, one giant leap for mankind" marked a significant achievement in space exploration', 'Taylor Swift', '

## 프롬프트 HotpotVer & MulithopRAG ver
hotpot과 Multihoop 각각 프롬프트를 다르게 줘서 해당하는 것 진행할때 아래에서 가져다 쓰면 됩니다.

In [ ]:
#######################MultihopRAG#####################
# 서브쿼리 생성 프롬프트
def generate_subqueries2(original_query, split_results):
    prompt = f"""
Given a query with multiple entities or phrases separated by conjunctions or commas, create each subquery by keeping as much of the original phrasing as necessary to maintain the intent and meaning of the original query.

Instructions:
1. For each phrase or section after a conjunction or comma, create a complete subquery that retains necessary context from the original query.
2. Ensure each subquery stands alone with full context so no information or intent is lost, and no subquery is overly simplified.
3. Maintain original terms where possible and ensure each subquery is grammatically complete.

Original Query: "{original_query}"
Separated Parts:
{split_results}

Example:
Original Query: "What is the name of the organization discussed in TechCrunch articles that, despite its financial instability, is recognized for creating ChatGPT, which is both a priority and a platform for ongoing innovations, and is planning to enhance its capabilities with the release of GPT-4 and associated APIs?"
->
- subquery: What is the name of the organization recognized for creating ChatGPT?
- subquery: What is the name of the organization recognized for creating ChatGPT, which is both a priority and a platform for ongoing innovations?
- subquery: What is the name of the organization planning to enhance ChatGPT with the release of GPT-4 and associated APIs?
- subquery: What is the name of the organization discussed in TechCrunch articles, despite its financial instability?

Example:
Original Query: "profit of Apple, Tesla and Microsoft?"
->
- subquery: profit of Apple
- subquery: profit of Tesla
- subquery: profit of Microsoft

Another Example:
Original Query: "Which company, as reported by both TechCrunch and The Verge, has spent billions to maintain its default search engine status on various platforms and is also accused of harming news publishers’ revenue through its business practices?"
- subquery: Which company has spent billions to maintain its default search engine status on various platforms?
- subquery: Which company is accused of harming news publishers’ revenue through its business practices?
- subquery: Which company is reported by both TechCrunch and The Verge for these actions?

One more example:
Original Query: "Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by both The Verge and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?"
->
- subquery: Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud?
- subquery: Who is the individual associated with the cryptocurrency industry facing a criminal trial conspiracy charges?
- subquery: Who is the individual associated with the cryptocurrency industry as reported by The Verge?
- subquery: Who is the individual associated with the cryptocurrency industry as reported by TechCrunch?
- subquery: Who is the individual is accused by prosecutors of committing fraud for personal gain?

Now, based on the above examples, provide a list of subqueries for each part of the question while preserving the essential meaning.
- subquery: <- Don't say anything other than the format that starts with this form. And the decomposed query that is generated can't just generate the original question. Don't forget the purpose of dividing in the original question.
"""
    return prompt

In [ ]:
#######################MultihopRAG22222222222222222222#####################
# 서브쿼리 생성 프롬프트
def generate_subqueries2(original_query, split_results):
    prompt = f"""
Given a query with multiple entities or phrases separated by conjunctions or commas, create each subquery by keeping as much of the original phrasing as necessary to maintain the intent and meaning of the original query.

Instructions:
1. Look for an omission at the beginning of the current section to create a full subquery that maintains the context required in the original query for each phrase or section that follows an conjunction or comma.
2. Ensure that each subquery stands independently with comprehensive context, preventing any loss of information or intent, and avoiding excessive simplification.
3. Preserve original terminology wherever possible, guaranteeing that each subquery forms a grammatically complete sentence.

Original Query: "{original_query}"
Separated Parts:
{split_results}

Example:
Original Query: "What is the name of the organization discussed in TechCrunch articles that, despite its financial instability, is recognized for creating ChatGPT, which is both a priority and a platform for ongoing innovations, and is planning to enhance its capabilities with the release of GPT-4 and associated APIs?"
Seperated Parts: ['What is the name of the organization discussed in TechCrunch articles that', 'despite its financial instability' , 'is recognized for creating ChatGPT', 'which is both a priority and a platform for ongoing innovations', 'is planning to enhance its capabilities with the release of GPT-4', 'associated APIs?']
->
- subquery: What is the name of the organization discussed in TechCrunch articles, despite its financial instability?
- subquery: What is the name of the organization recognized for creating ChatGPT?
- subquery: What is the name of the organization recognized for creating ChatGPT, which is both a priority and a platform for ongoing innovations?
- subquery: What is the name of the organization planning to enhance ChatGPT with the release of GPT-4?
- subquery: What is the name of the organization planning to enhance ChatGPT associated APIs?

Example:
Original Query: "profit of Apple, Tesla and Microsoft?"
Seperated Parts: ['profit of Apple', 'Tesla', 'Microsoft?']
->
- subquery: profit of Apple
- subquery: profit of Tesla
- subquery: profit of Microsoft


One more example:
Original Query: "Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by both The Verge and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?"
Seperated Parts: ['Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud', 'conspiracy charges', 'as reported by both The Verge and TechCrunch', 'is accused by prosecutors of committing fraud for personal gain?']
->
- subquery: Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud?
- subquery: Who is the individual associated with the cryptocurrency industry facing a criminal trial conspiracy charges?
- subquery: Who is the individual associated with the cryptocurrency industry as reported by both The Verge and TechCrunch?
- subquery: Who is the individual is accused by prosecutors of committing fraud for personal gain?


Now, based on the above examples, provide a list of subqueries for each part of the question while preserving the essential meaning.
Don't say anything other than the format that starts with this form (- subquery: ). And the decomposed query that is generated can't just generate the original question. Don't forget the purpose of dividing in the original question.
"""
    return prompt

In [ ]:
#######################MultihopRAG333333333333#####################
# 서브쿼리 생성 프롬프트
def generate_subqueries2(original_query, split_results):
    prompt = f"""
Given a query with multiple entities or phrases separated by conjunctions or commas, create each subquery by keeping as much of the original phrasing as necessary to maintain the intent and meaning of the original query.

Instructions:
1. For each separated part, complete the meaning of the question using the previous part of the current section.
2. Preserve original terminology wherever possible, guaranteeing that each subquery forms a grammatically complete sentence.
3. The number of sub-questions created must match the number of separated parts.

Original Query: "{original_query}"
Separated Parts:
{split_results}

Example:
Original Query: "What is the name of the organization discussed in TechCrunch articles that, despite its financial instability, is recognized for creating ChatGPT, which is both a priority and a platform for ongoing innovations, and is planning to enhance its capabilities with the release of GPT-4 and associated APIs?"
Seperated Parts: ['What is the name of the organization discussed in TechCrunch articles that', 'despite its financial instability' , 'is recognized for creating ChatGPT', 'which is both a priority and a platform for ongoing innovations', 'is planning to enhance its capabilities with the release of GPT-4', 'associated APIs?']
->
- subquery: What is the name of the organization discussed in TechCrunch articles?
- subquery: What is the name of the organization discussed in TechCrunch articles, despite its financial instability?
- subquery: What is the name of the organization recognized for creating ChatGPT?
- subquery: What is the name of the organization recognized for creating ChatGPT, which is both a priority and a platform for ongoing innovations?
- subquery: What is the name of the organization planning to enhance ChatGPT with the release of GPT-4?
- subquery: What is the name of the organization planning to enhance ChatGPT associated APIs?

Example:
Original Query: "what is profit of Apple, Tesla and Microsoft and non-profit of Nexans?"
Seperated Parts: ['what is profit of Apple', 'Tesla', 'Microsoft','non-profit of Nexans?']
->
- subquery: what is profit of Apple?
- subquery: what is profit of Tesla?
- subquery: what is profit of Microsoft?
- subquery: what is non-profit of Nexans?


One more example:
Original Query: "Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by both The Verge and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?"
Seperated Parts: ['Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud', 'conspiracy charges', 'as reported by both The Verge and TechCrunch', 'is accused by prosecutors of committing fraud for personal gain?']
->
- subquery: Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud?
- subquery: Who is the individual associated with the cryptocurrency industry facing a criminal trial conspiracy charges?
- subquery: Who is the individual associated with the cryptocurrency industry as reported by both The Verge and TechCrunch?
- subquery: Who is the individual is accused by prosecutors of committing fraud for personal gain?


Now, based on the above examples, provide a list of subqueries for each part of the question while preserving the essential meaning.
Don't say anything other than the format that starts with this form (- subquery: ). And the decomposed query that is generated can't just generate the original question. Don't forget the purpose of dividing in the original question.
"""
    return prompt

In [ ]:
#######안쓸거같긴한데
#######################MultihopRAG#####################
# 서브쿼리 생성 프롬프트
def generate_subqueries2(original_query, split_results):
    prompt = f"""
Given a query with multiple entities or phrases separated by conjunctions or commas, create each subquery by keeping as much of the original phrasing as necessary to maintain the intent and meaning of the original query.

Instructions:
1. For each separated part, complete the meaning of the question using the previous part of the current section.
2. Preserve original terminology wherever possible, guaranteeing that each subquery forms a grammatically complete sentence.
3. The number of sub-questions created must match the number of separated parts.

Original Query: "{original_query}"
Separated Parts:
{split_results}


Example:
Original Query: "~~A, ~~~B ~~~ and ~~C ~~?"
Seperated Parts: ['~~A', '~~~B ~~~', '~~C ~~?']
->
- subquery: subqforA
- subquery: subqforB
- subquery: subqforC

Now, based on the above examples, provide a list of subqueries for each part of the question while preserving the essential meaning.
Don't say anything other than the format that starts with this form (- subquery: ). And the decomposed query that is generated can't just generate the original question. Don't forget the purpose of dividing in the original question.
"""
    return prompt

In [ ]:
#################hotpotQA############################
# 서브쿼리 생성 프롬프트
def generate_subqueries2(original_query, split_results):
    prompt = f"""
Given a query with multiple entities or phrases separated by conjunctions or commas, create each subquery by keeping as much of the original phrasing as necessary to maintain the intent and meaning of the original query.

Instructions:
1. For each phrase or section after a conjunction or comma, create a complete subquery that retains necessary context from the original query.
2. Ensure each subquery stands alone with full context so no information or intent is lost, and no subquery is overly simplified.
3. Maintain original terms where possible and ensure each subquery is grammatically complete.
When you fill in a sentence (subquery) in Separate Parts, always find it in the words in the original question and fill it in.


Original Query: "{original_query}"
Separated Parts:
{split_results}

Example:
Original Query: "Who has won more awards, Dan Schneider or Helen Hunt?"
->
- subquery: Who has won more awards than Helen Hunt?
- subquery: Who has won more awards than Dan Schneider?

Example:
Original Query: "profit of Apple, Tesla and Microsoft?"
->
- subquery: profit of Apple
- subquery: profit of Tesla
- subquery: profit of Microsoft

One more example:
Original Query: "Which American singer and songwriter has a mezzo-soprano vocal range, Tim Armstrong or Tori Amos?"
->
- subquery: Which American singer has a mezzo-soprano vocal range?
- subquery: Which American songwriter has a mezzo-soprano vocal range?
- subquery: Is Tim Armstrong an American singer and songwriter with a mezzo-soprano vocal range?
- subquery: Is Tori Amos an American singer and songwriter with a mezzo-soprano vocal range?

Now, based on the above examples, provide a list of subqueries for each part of the question while preserving the essential meaning.
- subquery: <- Don't say anything other than the format that starts with this form. And the decomposed query that is generated can't just generate the original question. Don't forget the purpose of dividing in the original question.
"""
    return prompt

In [ ]:
#################RE####################
# 서브쿼리 생성 프롬프트
def generate_subqueries2(original_query, split_results):
    prompt = f"""
Given a query with multiple entities or phrases separated by conjunctions or commas, create each subquery by keeping as much of the original phrasing as necessary to maintain the intent and meaning of the original query.

Instructions:
1. For each phrase or section after a conjunction or comma, create a complete subquery that retains necessary context from the original query.
2. Ensure each subquery stands alone with full context so no information or intent is lost, and no subquery is overly simplified.
3. Maintain original terms where possible and ensure each subquery is grammatically complete.

Original Query: "{original_query}"
Separated Parts:
{split_results}

Example:
Original Query: "Who's birth date is the first earliest, among the birth date of Neil Armstrong, the pioneering astronaut who became the first human to set foot on the moon during NASA's Apollo 11 mission in 1969, and Taylor Swift and Vladimir Putin?"
Seperated Parts: ['Who's birth date is the first earliest', 'among the birth date of Neil Armstrong', 'the pioneering astronaut who became the first human to set foot on the moon during NASA's Apollo 11 mission in 1969' ,'Taylor Swift' ,'Vladimir Putin?' ]
->
- subquery: Who's birth date is the first earliest, among the birth date of Neil Armstrong who is the pioneering astronaut who became the first human to set foot on the moon during NASA's Apollo 11 mission in 1969?
- subquery: Who's birth date is the first earliest, among the birth date of Taylor Swift?
- subquery: Who's birth date is the first earliest, among the birth date of Vladimir Putin?

Example:
Original Query: "profit of Apple, Tesla and Microsoft?"
Seperated Parts: ['profit of Apple', 'Tesla', 'Microsoft?']
->
- subquery: profit of Apple
- subquery: profit of Tesla
- subquery: profit of Microsoft

Example:
Original Query: "What is the name of the organization discussed in TechCrunch articles that, despite its financial instability, is recognized for creating ChatGPT, which is both a priority and a platform for ongoing innovations, and is planning to enhance its capabilities with the release of GPT-4 and associated APIs?"
Seperated Parts: ['What is the name of the organization discussed in TechCrunch articles that', 'despite its financial instability' , 'is recognized for creating ChatGPT', 'which is both a priority and a platform for ongoing innovations', 'is planning to enhance its capabilities with the release of GPT-4', 'associated APIs?']
->
- subquery: What is the name of the organization discussed in TechCrunch articles, despite its financial instability?
- subquery: What is the name of the organization recognized for creating ChatGPT?
- subquery: What is the name of the organization recognized for creating ChatGPT, which is both a priority and a platform for ongoing innovations?
- subquery: What is the name of the organization planning to enhance ChatGPT with the release of GPT-4?
- subquery: What is the name of the organization planning to enhance ChatGPT associated APIs?


Now, based on the above examples, provide a list of subqueries for each part of the question while preserving the essential meaning.
- subquery: <- Don't say anything other than the format that starts with this form. And the decomposed query that is generated can't just generate the original question. Don't forget the purpose of dividing in the original question.
"""
    return prompt

In [ ]:
# LLM 호출 예시
def get_subqueries2(original_query, model="gpt-4o"):
    original_query = "Who's birth date is the first earliest, among the birth date of Neil Armstrong, the first human to set foot on the Moon during the Apollo 11 mission in 1969, whose historic words, \"That's one small step for a man, one giant leap for mankind,\"marked a significant achievement in space exploration, Taylor Swift, and Chris Hemsworth?"
    split_results = "['Who's birth date is the first earliest', 'among the birth date of Neil Armstrong', 'the first human to set foot on the Moon during the Apollo 11 mission in 1969', 'whose historic words, \"That\'s one small step for a man, one giant leap for mankind\"', 'marked a significant achievement in space exploration', 'Taylor Swift', 'Chris Hemsworth?']"
    #split_results = [part.strip() for part in split_results_input.split(",") if part.strip()]

    prompt = generate_subqueries2(original_query, split_results)
    print(prompt)
    response = openai.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    # 각 서브쿼리의 앞에 있는 "subquery:"와 "-"를 제거하고 리스트로 반환
    subqueries = [
        query.replace("- subquery: ", "").strip()  # "subquery:" 제거
        for query in response.choices[0].message.content.splitlines() if query.strip()
    ]
    return subqueries